In [1]:
import numpy as np
import nnpy

In [2]:
# create network
net = [
    nnpy.Flatten(keep_channels=True),
    nnpy.Linear(28*28, 128),
    nnpy.ReLU(),
    nnpy.BatchNorm(1),
    nnpy.Linear(128, 64),
    nnpy.Sigmoid(),
    nnpy.Flatten(),
    nnpy.Linear(64, 10)
]

In [3]:
net

In [4]:
# create loss function
criterion = nnpy.CrossEntropy()

# create optimizer
sgd = nnpy.SGD(net, learn_rate=0.01, momentum=0.01)

In [5]:
from keras.datasets import mnist

# create data
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# normalize
X_train = X_train.astype(np.float32) / 255
X_test = X_test.astype(np.float32) / 255

# make 4d
X_train = np.expand_dims(X_train, axis=1)
X_test = np.expand_dims(X_test, axis=1)

Using TensorFlow backend.


In [6]:
def train():
    epoch_loss = []
    epoch_predictions = []
    epoch_labels = []

    nnpy.set_train(net)

    for x_batch, y_batch in nnpy.dataloader(X_train, y_train, batchsize=128, shuffle=True):
        logits = nnpy.forward(net, x_batch)
        loss_value = criterion.forward(logits, y_batch).mean()
        nnpy.backward(net, criterion)
        sgd.step()

        epoch_loss.append(loss_value)
        epoch_predictions.append(np.argmax(logits, axis=-1))
        epoch_labels.append(y_batch)

    epoch_predictions = np.array(epoch_predictions).flatten()
    epoch_labels = np.array(epoch_labels).flatten()

    return epoch_loss, epoch_predictions, epoch_labels

In [7]:
def test():
    epoch_loss = []
    epoch_predictions = []
    epoch_labels = []

    nnpy.set_eval(net)

    for x_batch, y_batch in nnpy.dataloader(X_test, y_test, batchsize=128, shuffle=False):
        logits = nnpy.forward(net, x_batch)
        loss_value = criterion.forward(logits, y_batch).mean()
        nnpy.backward(net, criterion)
        sgd.step()

        epoch_loss.append(loss_value)
        epoch_predictions.append(np.argmax(logits, axis=-1))
        epoch_labels.append(y_batch)

    epoch_predictions = np.array(epoch_predictions).flatten()
    epoch_labels = np.array(epoch_labels).flatten()

    return epoch_loss, epoch_predictions, epoch_labels

In [8]:
epochs = 10
for e in range(epochs):

    epoch_loss_train, epoch_predictions_train, epoch_labels_train = train()
    epoch_loss_test, epoch_predictions_test, epoch_labels_test = test()

    print(f"epoch: {e:3d},",
          f"train loss: {np.mean(epoch_loss_train):.4f},",
          f"train acc: {nnpy.accuracy(epoch_predictions_train, epoch_labels_train):.4f},",
          f"test loss: {np.mean(epoch_loss_test):.4f},",
          f"test acc: {nnpy.accuracy(epoch_predictions_test, epoch_labels_test):.4f}"
          )

epoch:   0, train loss: 2.2938, train acc: 0.1138, test loss: 2.2680, test acc: 0.1352
epoch:   1, train loss: 2.0723, train acc: 0.3539, test loss: 1.6417, test acc: 0.5170
epoch:   2, train loss: 1.3242, train acc: 0.6013, test loss: 1.1175, test acc: 0.6638
epoch:   3, train loss: 1.1360, train acc: 0.6657, test loss: 1.0251, test acc: 0.6934
epoch:   4, train loss: 0.9824, train acc: 0.7010, test loss: 0.9103, test acc: 0.7290
epoch:   5, train loss: 0.8856, train acc: 0.7306, test loss: 0.8303, test acc: 0.7518
epoch:   6, train loss: 0.8147, train acc: 0.7532, test loss: 0.7745, test acc: 0.7682
epoch:   7, train loss: 0.7620, train acc: 0.7688, test loss: 0.7277, test acc: 0.7812
epoch:   8, train loss: 0.7222, train acc: 0.7807, test loss: 0.6917, test acc: 0.7934
epoch:   9, train loss: 0.6890, train acc: 0.7922, test loss: 0.6625, test acc: 0.8014
